In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
import re
import time

start_time = time.time()



In [2]:
browser = Browser('chrome','executable_path'== 'chromedriver', headless=True)
url = 'https://reelgood.com/movies'

browser.visit(url)

In [3]:
file = 'Missing_IMDb.csv'

missing_imdb = pd.read_csv(file)

In [4]:
sample = missing_imdb.copy()
print(len(sample))
sample

571


,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,Making Unorthodox,2020,NaN,NaN,NaN,1,0,0,0,0,Marlene Melchior,"Short,Documentary",Germany,English,NaN
1,Little Singham aur Kaal ka Mahajaal,2018,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,Night on Earth: Shot in the Dark,2020,NaN,NaN,NaN,1,0,0,0,0,Alex Minton,Documentary,NaN,NaN,59.0
3,What the F* Is Going On?,2019,NaN,NaN,NaN,1,0,0,0,0,Bob Spiers,Comedy,NaN,NaN,NaN
4,Uncovered,2019,NaN,NaN,NaN,1,0,0,0,0,Jim McBride,Thriller,"United Kingdom,Spain,France",English,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,Cracking the Prophetic Code,2000,NaN,NaN,NaN,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN
567,Behind the Race to the Moon,2002,NaN,NaN,NaN,0,0,1,0,0,NaN,NaN,NaN,NaN,51.0
568,Late War Fighters of the Luftwaffe,1997,NaN,NaN,NaN,0,0,1,0,0,NaN,"Documentary,History,War",NaN,NaN,45.0
569,Beneath Alcatraz,1999,NaN,NaN,NaN,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN


In [6]:
def src_movie_data(search_num):

    global data
    global imdb
    global age_rating
    global dir_names
    global runtime
    global genres
    global countries
    
    actual_title = sample['Title'].tolist()[search_num]
    search_title = re.sub('[^A-Za-z0-9 ]+', '', actual_title)
    search_year = str(sample['Year'].tolist()[search_num])

    new_url = 'https://reelgood.com/movie'

    movie_src = search_title.replace(' ', '-').replace(':', '').lower()
    print(search_num, search_title)
    movie_url = new_url + '/' + movie_src + '-' + search_year
    browser.visit(movie_url)

    html = browser.html

    global new_soup
    new_soup = soup(html, 'html.parser')

    check_404 = new_soup.find(class_='css-7fo8ar e1oyx0om3')
    print(movie_url)

    if check_404 is not None:
        print('movie not found... skipping... ')
        imdb = 'NaN'
        dir_names = 'NaN'
        runtime = 'NaN'
        genres = 'NaN'
        countries = 'NaN'
        age_rating = 'NaN'
        
        
    

    else:
        data = new_soup.find_all('span', class_='css-dpg6zk ey4ir3j1')
        get_info(new_soup)
        
#     print(search_title, imdb, age_rating, dir_names, runtime, genres, countries)
    print(f"-----------------------------")
    movies_list.append(actual_title)
    imdb_list.append(imdb)
    age_list.append(age_rating)
    directors_list.append(dir_names)
    runtime_list.append(runtime)
    plat_netflix.append(netflix_plat)
    plat_hulu.append(hulu_plat)
    plat_prime.append(prime_plat)
    plat_disney.append(disney_plat)
    genres_list.append(genres)
    countries_list.append(countries)
    
#     new_df = {'movies': movies_list,
#              'imdb': imdb_list,
#              'runtime': runtime_list,
#              'directors': dir_names,
#              'Netflix': plat_netflix,
#              'Hulu': plat_hulu,
#               'Prime': plat_prime,
#               'Disney+': plat_disney,
#              'genres': genres_list,
#              'countries': countries_list}
    
#     return new_df
    

    


In [7]:
# get imdb rating
def get_info(new_soup):
    
    data = new_soup.find_all('span', class_='css-dpg6zk ey4ir3j1')
    
    global imdb
    global age_rating
    global runtime
    global genres
    global countries
    global netflix_plat
    global hulu_plat
    global prime_plat
    global disney_plat
    global dir_names

    imdb = 'NaN'
    age_rating = 'NaN'
    runtime = 'NaN'
    genres = 'NaN'
    countries = 'NaN'

    
    for i in range(len(data)):
                
        # Look for IMDb Rating
        if '/' in data[i].text:
            
            imdb = data[i].text.split('/')[0]
            
        # -----------------------------------
        
        elif 'Rated' in str(data[i].text):
#             print(i)

            age_rating = data[i].text.split()[1]
            
        
        # -----------------------------------
        # Look for Runtimes
        elif len(data[i].text) < 12 and ' ' in data[i].text and 'm' in data[i].text:


            if 'h' in data[i].text:
                    h, m = data[i].text.split()
                    hr_min = float(re.sub('[^0-9]+', '', h))*60
                    mins = float(re.sub('[^0-9]+', '', m))
                    runtime = hr_min + mins

            else:

                    runtime = float(data[i].text.split()[0])
                    
        # --------------------------------------------------
        
        # Look for Genres
        elif 'genre' in str(data[i]):

            global test
            genres_add = []

            test = new_soup.find_all('div', class_='css-19fr2c5')

            if len(test) != 0:
                more_genre = test[0].find_all('a')

                for i in range(len(more_genre)):
                    genres_add.append(more_genre[i].text)
                    genres = ', '.join(genres_add)

            else:
                test = data[i].find_all('a', class_='css-10wrqt0')

                for i in range(len(test)):
                    genres_add.append(test[i].text)
                    genres = ', '.join(genres_add)
        
        # ---------------------------------------------------
        
        # Look for Countries
        elif 'Country' in str(data[i]):
            
            print(i)
            global ctry_src
            countries_add = []


            ctry_src = data[i].find_all('a', class_='css-10wrqt0')

            for i in range(len(ctry_src)):
                print(ctry_src[i].text)
                countries_add.append(ctry_src[i].text)
                countries = ', '.join(countries_add)    
    
        # ---------------------------------------------------
        
    plat_info = str(new_soup.find_all('a', class_='css-10wrqt0'))
    netflix_plat = 0
    hulu_plat = 0
    prime_plat = 0
    disney_plat = 0
    
    if 'Netflix' in plat_info:
        netflix_plat = 1
                
    if 'Hulu' in plat_info:
        hulu_plat = 1
        
    if 'Prime' in plat_info:
        prime_plat = 1
        
    if 'Disney' in plat_info:
        disney_plat = 1
        
                    
    director_data = new_soup.find_all('a', itemprop='director')

    if len(director_data) == 0:
        dir_names = 'NaN'
        
    else:
        directors = []
        for i in range(len(director_data)):

            dir_name = director_data[i].find('h3', itemprop='name').text
            directors.append(dir_name)
            dir_names = ', '.join(directors)
            

    

    


In [9]:
movies_list = []
imdb_list = []
age_list = []
directors_list = []
runtime_list = []
plat_netflix = []
plat_hulu = []
plat_prime = []
plat_disney = []
genres_list = []
countries_list = []

for i in range(len(sample)):
# for i in range(50):
    src_movie_data(i)

end_time = time.time()

print(f"Scraping took {(end_time - start_time)/60} minutes")


updated_df = pd.DataFrame({'Title': movies_list,
                              'IMDb': imdb_list,
                               'Age': age_list,
                             'Directors': directors_list,
                            'Runtime': runtime_list,
                            'Netflix': plat_netflix,
                           'Hulu': plat_hulu,
                           'Prime Video': plat_prime,
                           'Disney+': plat_disney,
                            'Genres': genres_list,
                          'Country': countries_list})

updated_df
# browser.quit()

0 Making Unorthodox
https://reelgood.com/movie/making-unorthodox-2020
-----------------------------
1 Little Singham aur Kaal ka Mahajaal
https://reelgood.com/movie/little-singham-aur-kaal-ka-mahajaal-2018
movie not found... skipping... 
-----------------------------
2 Night on Earth Shot in the Dark
https://reelgood.com/movie/night-on-earth-shot-in-the-dark-2020
-----------------------------
3 What the F Is Going On
https://reelgood.com/movie/what-the-f-is-going-on-2019
-----------------------------
4 Uncovered
https://reelgood.com/movie/uncovered-2019
5
South Africa
-----------------------------
5 Concrete Football
https://reelgood.com/movie/concrete-football-2016
-----------------------------
6 Luccas Neto in Summer Camp
https://reelgood.com/movie/luccas-neto-in-summer-camp-2019
-----------------------------
7 Alexis Viera A Story of Surviving
https://reelgood.com/movie/alexis-viera-a-story-of-surviving-2019
-----------------------------
8 Amelia A Tale of Two Sisters
https://reelgo

https://reelgood.com/movie/lorai-play-to-live-2015
-----------------------------
64 Mission Destroy Love
https://reelgood.com/movie/mission-destroy-love-2014
-----------------------------
65 Oh My Ghost 4
https://reelgood.com/movie/oh-my-ghost-4-2015
-----------------------------
66 Sohni Mahiwal
https://reelgood.com/movie/sohni-mahiwal-1984
-----------------------------
67 Twice
https://reelgood.com/movie/twice-2010
-----------------------------
68 Karol Modzelewski ukasz Lotek Lodkowski No Offense
https://reelgood.com/movie/karol-modzelewski-ukasz-lotek-lodkowski-no-offense-2016
movie not found... skipping... 
-----------------------------
69 Maj Rati Keteki
https://reelgood.com/movie/maj-rati-keteki-2017
-----------------------------
70 Paradise Lost
https://reelgood.com/movie/paradise-lost-2017
movie not found... skipping... 
-----------------------------
71 Monty Python Conquers America
https://reelgood.com/movie/monty-python-conquers-america-2000
-----------------------------
72 

https://reelgood.com/movie/legend-of-heroes-trails-in-the-sky-2011
-----------------------------
128 Hannah
https://reelgood.com/movie/hannah-2018
-----------------------------
129 The White Massai Warrior
https://reelgood.com/movie/the-white-massai-warrior-2020
-----------------------------
130 Mothers Day
https://reelgood.com/movie/mothers-day-2019
-----------------------------
131 Afterlife
https://reelgood.com/movie/afterlife-2018
-----------------------------
132 Abbott and Costello Christmas Show
https://reelgood.com/movie/abbott-and-costello-christmas-show-1952
-----------------------------
133 Buffed Up
https://reelgood.com/movie/buffed-up-2016
-----------------------------
134 Leftovers
https://reelgood.com/movie/leftovers-2017
-----------------------------
135 Vintage Sinatra
https://reelgood.com/movie/vintage-sinatra-2003
-----------------------------
136 Too Cold to Swim
https://reelgood.com/movie/too-cold-to-swim-2018
-----------------------------
137 OCD and Me
https://re

https://reelgood.com/movie/soul-2020
-----------------------------
197 Curse of the Dragon Slayer
https://reelgood.com/movie/curse-of-the-dragon-slayer-2013
-----------------------------
198 Bones of Contention
https://reelgood.com/movie/bones-of-contention-1994
movie not found... skipping... 
-----------------------------
199 The Getaway
https://reelgood.com/movie/the-getaway-2020
movie not found... skipping... 
-----------------------------
200 Drone
https://reelgood.com/movie/drone-2014
6
Norway
-----------------------------
201 Acrylic
https://reelgood.com/movie/acrylic-2020
-----------------------------
202 AI Artificial or ADVANCED Intelligence
https://reelgood.com/movie/ai-artificial-or-advanced-intelligence-2020
-----------------------------
203 A risky revenge
https://reelgood.com/movie/a-risky-revenge-2019
-----------------------------
204 The Daniel Tiger Movie Wont You Be Our Neighbor
https://reelgood.com/movie/the-daniel-tiger-movie-wont-you-be-our-neighbor-2018
----------

https://reelgood.com/movie/is-sugar-the-new-fat-2014
-----------------------------
262 When Darkness Falls 2 Chiller Tales
https://reelgood.com/movie/when-darkness-falls-2-chiller-tales-2017
-----------------------------
263 The Fabulous Life Of Elisabeth Vigee Labrun
https://reelgood.com/movie/the-fabulous-life-of-elisabeth-vigee-labrun-2018
-----------------------------
264 Ancient Amazon
https://reelgood.com/movie/ancient-amazon-1999
-----------------------------
265 Luger Pistols
https://reelgood.com/movie/luger-pistols-2018
-----------------------------
266 A Christmas Carol in Color
https://reelgood.com/movie/a-christmas-carol-in-color-2008
-----------------------------
267 The Middle Man
https://reelgood.com/movie/the-middle-man-2018
-----------------------------
268 WWII Bomber Boys of Britain with Ewan McGregor
https://reelgood.com/movie/wwii-bomber-boys-of-britain-with-ewan-mcgregor-2014
-----------------------------
269 Bedtime Lullabies
https://reelgood.com/movie/bedtime-lu

https://reelgood.com/movie/arthur-and-the-haunted-tree-house-2017
-----------------------------
328 OGAR Will of Steel
https://reelgood.com/movie/ogar-will-of-steel-2017
-----------------------------
329 Kurdish Factor The Untold Story Of The Gulf Wars
https://reelgood.com/movie/kurdish-factor-the-untold-story-of-the-gulf-wars-2018
-----------------------------
330 The Story of Queen Elizabeth II
https://reelgood.com/movie/the-story-of-queen-elizabeth-ii-2002
-----------------------------
331 On The Court
https://reelgood.com/movie/on-the-court-2017
-----------------------------
332 Bonobo  The North Borders Tour  Live
https://reelgood.com/movie/bonobo--the-north-borders-tour--live-2014
movie not found... skipping... 
-----------------------------
333 Have Fun
https://reelgood.com/movie/have-fun-2018
-----------------------------
334 My Phamily BBQ
https://reelgood.com/movie/my-phamily-bbq-2003
-----------------------------
335 The Battle Of Bunker Hill
https://reelgood.com/movie/the-b

https://reelgood.com/movie/paranormal-rosslyn-chapel-2013
-----------------------------
392 Robinson Crusoes Treasure Island
https://reelgood.com/movie/robinson-crusoes-treasure-island-2011
-----------------------------
393 Chonda Pierce Presents Stand Up for Families  Home Is Where the Heart Is
https://reelgood.com/movie/chonda-pierce-presents-stand-up-for-families--home-is-where-the-heart-is-2016
movie not found... skipping... 
-----------------------------
394 Jason Mraz Have It All The Movie
https://reelgood.com/movie/jason-mraz-have-it-all-the-movie-2018
-----------------------------
395 Hitlers Secret Son
https://reelgood.com/movie/hitlers-secret-son-2018
-----------------------------
396 Jurassic Africa
https://reelgood.com/movie/jurassic-africa-2018
-----------------------------
397 In From The Cold A Portrait Of Richard Burton
https://reelgood.com/movie/in-from-the-cold-a-portrait-of-richard-burton-1988
-----------------------------
398 Third Day
https://reelgood.com/movie/thi

https://reelgood.com/movie/it-takes-guts-2016
-----------------------------
462 Edith Wharton The Sense of Harmony
https://reelgood.com/movie/edith-wharton-the-sense-of-harmony-2014
-----------------------------
463 Guns of the Civil War
https://reelgood.com/movie/guns-of-the-civil-war-2018
-----------------------------
464 Set Free
https://reelgood.com/movie/set-free-2017
-----------------------------
465 The Iranian Americans
https://reelgood.com/movie/the-iranian-americans-2012
-----------------------------
466 Profit Pollution and Deception
https://reelgood.com/movie/profit-pollution-and-deception-2010
-----------------------------
467 Who Put The Klan Into Ku Klux Klan
https://reelgood.com/movie/who-put-the-klan-into-ku-klux-klan-2018
-----------------------------
468 Las Vegas Story
https://reelgood.com/movie/las-vegas-story-2015
-----------------------------
469 The Healing
https://reelgood.com/movie/the-healing-1988
-----------------------------
470 New York Murder
https://reel

https://reelgood.com/movie/diary-of-a-cheating-man-2015
-----------------------------
534 Kimono
https://reelgood.com/movie/kimono-2007
-----------------------------
535 TurboDojo  Absinthe Films
https://reelgood.com/movie/turbodojo--absinthe-films-2017
movie not found... skipping... 
-----------------------------
536 Snowsurf
https://reelgood.com/movie/snowsurf-2015
-----------------------------
537 Laughter Is Medicine
https://reelgood.com/movie/laughter-is-medicine-2018
-----------------------------
538 Evening Waterfalls 8 hours for sleep and meditation ultra dark
https://reelgood.com/movie/evening-waterfalls-8-hours-for-sleep-and-meditation-ultra-dark-2016
-----------------------------
539 Goonside 2
https://reelgood.com/movie/goonside-2-2018
-----------------------------
540 Flexibility Yoga 30 Minute Workout To Boost Mobility  Julia Jarvis
https://reelgood.com/movie/flexibility-yoga-30-minute-workout-to-boost-mobility--julia-jarvis-2017
movie not found... skipping... 
----------

,Title,IMDb,Age,Directors,Runtime,Netflix,Hulu,Prime Video,Disney+,Genres,Country
0,Making Unorthodox,NaN,NaN,Marlene Melchior,21,1,0,0,0,"Drama, Documentary",NaN
1,Little Singham aur Kaal ka Mahajaal,NaN,NaN,NaN,NaN,1,0,0,0,NaN,NaN
2,Night on Earth: Shot in the Dark,8.2,NaN,Alex Minton,59,1,0,0,0,Documentary,NaN
3,What the F* Is Going On?,7.6,NaN,"Marta Jaenes, Rosa Marquez",86,0,0,0,0,Documentary,NaN
4,Uncovered,NaN,NaN,Zuko Nodada,95,1,0,0,0,"Drama, Crime, Thriller",South Africa
...,...,...,...,...,...,...,...,...,...,...,...
566,Cracking the Prophetic Code,NaN,NaN,NaN,NaN,0,0,1,0,NaN,NaN
567,Behind the Race to the Moon,NaN,NaN,NaN,90,0,0,0,0,NaN,NaN
568,Late War Fighters of the Luftwaffe,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN
569,Beneath Alcatraz,NaN,NaN,NaN,NaN,0,0,1,0,NaN,NaN


In [10]:
non_nan_df = updated_df[updated_df['IMDb'] != 'NaN']
print(len(non_nan_df))
non_nan_df['Runtime'] = non_nan_df['Runtime'].astype(float)
non_nan_df.to_csv('updated_imdb.csv', index=False)
non_nan_df

49


C:\Users\daytw\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Title,IMDb,Age,Directors,Runtime,Netflix,Hulu,Prime Video,Disney+,Genres,Country
2,Night on Earth: Shot in the Dark,8.2,NaN,Alex Minton,59.0,1,0,0,0,Documentary,NaN
3,What the F* Is Going On?,7.6,NaN,"Marta Jaenes, Rosa Marquez",86.0,0,0,0,0,Documentary,NaN
5,Concrete Football,6.4,NaN,"Jesse Adang, Syrine Boulanouar",51.0,0,0,0,0,"Documentary, Sport",NaN
7,Alexis Viera: A Story of Surviving,7.7,NaN,Luis Ara,68.0,1,0,0,0,Documentary,NaN
23,I Am Jonas,7,NaN,NaN,82.0,1,0,0,0,"Drama, Mystery, Romance",NaN
25,Light in the Dark,6.1,16+,Ekene Som Mekwunye,90.0,1,0,0,0,"Thriller, Drama",NaN
26,Patton Oswalt: I Love Everything,6.4,NaN,NaN,90.0,1,0,0,0,Comedy,NaN
41,Homeland,5.3,NaN,Anindya Chatterjee,73.0,0,0,0,0,Drama,NaN
72,The Consul's Son,6.6,NaN,Amr Arafa,119.0,1,0,0,0,"Comedy, Thriller, Drama, Family",NaN
79,Little Baby Bum: Go Buster,6.4,NaN,NaN,NaN,0,0,0,0,"Animation, Family, Musical",NaN
